In [ ]:
import os
import numpy as np
import pandas as pd
import SimpleITK as sitk
from radiomics import featureextractor
import glob
from scipy.stats import kruskal
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LassoCV, Lasso, LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, StackingClassifier
from sklearn.metrics import (roc_auc_score, classification_report,
                             confusion_matrix, ConfusionMatrixDisplay,
                             accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc)
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.calibration import calibration_curve
import matplotlib.pyplot as plt
import xgboost as xgb
from imblearn.over_sampling import SMOTE  # 确保已安装imblearn库
from sklearn.model_selection import cross_val_score
import shap

# 设置随机种子，确保结果可复现
def seed_everything(seed=42):
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    # 如果使用PyTorch，也需要设置相关种子
    # torch.manual_seed(seed)
    # torch.cuda.manual_seed_all(seed)
    return seed

SEED = seed_everything(42)

# 定义训练集和测试集的图像和掩膜文件的文件夹路径
# 训练集路径
train_image_folder = "C:/Users/YK/Desktop/train_liver"
train_mask_folder = "C:/Users/YK/Desktop/train_liver-mask"

# 测试集路径
test_image_folder = "C:/Users/YK/Desktop/test_liver"
test_mask_folder = "C:/Users/YK/Desktop/test_liver-mask"

# 创建结果保存目录
results_dir = "LF-LML(E)"
os.makedirs(results_dir, exist_ok=True)

# 初始化特征提取器
extractor = featureextractor.RadiomicsFeatureExtractor()

# 启用所有标准特征类
feature_classes = [
    'firstorder', 'glcm', 'glrlm', 'glszm',
    'ngtdm', 'gldm', 'shape2D'  # 使用shape2D替代shape
]
for fc in feature_classes:
    extractor.enableFeatureClassByName(fc)

# 启用所有可用的图像类型
image_types = [
    'Wavelet', 'LoG', 'Square', 'SquareRoot',
    'Logarithm', 'Exponential', 'Gradient'
]
for it in image_types:
    extractor.enableImageTypeByName(it)

# 配置额外参数以增加特征
extractor.settings.update({
    'binWidth': 25,
    'normalize': True,
    'force2D': True  # 确保处理为2D图像
})

# 定义函数用于提取数据集特征
def extract_dataset_features(image_folder, mask_folder):
    features_list = []
    groups_list = []
    
    # 遍历文件夹中的文件
    for filename in os.listdir(image_folder):
        if filename.endswith('.jpg'):
            base_name = os.path.splitext(filename)[0]
            group_info = base_name.split('_')[0].split('F')[1]
            image_path = os.path.join(image_folder, filename)
            
            # 使用通配符匹配所有可能的掩膜文件
            mask_pattern = os.path.join(mask_folder, base_name + ".nrrd")
            mask_files = glob.glob(mask_pattern)
            
            for mask_path in mask_files:
                try:
                    # 读取图像和掩膜
                    image = sitk.ReadImage(image_path)
                    mask = sitk.ReadImage(mask_path)
                    
                    # 维度处理
                    if image.GetDimension() != mask.GetDimension():
                        if image.GetDimension() == 2 and mask.GetDimension() == 3:
                            mask = mask[:, :, 0]  # 取第一个切片
                        else:
                            print(f"维度不匹配已跳过：{image_path} | {mask_path}")
                            continue
                            
                    # 图像预处理
                    if image.GetNumberOfComponentsPerPixel() > 1:
                        image = sitk.VectorIndexSelectionCast(image, 0, sitk.sitkUInt8)
                    
                    # 特征提取
                    features = extractor.execute(image, mask)
                    
                    # 过滤诊断信息并保留所有特征
                    feature_values = {
                        k: v for k, v in features.items()
                        if not k.startswith('diagnostics')
                    }
                    
                    features_list.append(feature_values)
                    groups_list.append(group_info)
                    
                except Exception as e:
                    print(f"处理失败 {mask_path}: {str(e)}")
                    continue
    
    return features_list, groups_list

# 提取训练集特征
print("正在提取训练集特征...")
train_features_list, train_groups_list = extract_dataset_features(train_image_folder, train_mask_folder)

# 提取测试集特征
print("正在提取测试集特征...")
test_features_list, test_groups_list = extract_dataset_features(test_image_folder, test_mask_folder)

# 将特征转换为 DataFrame
X_train = pd.DataFrame(train_features_list)
X_test = pd.DataFrame(test_features_list)

# 将分组信息转换为 Series
y_train = pd.Series(train_groups_list)
y_test = pd.Series(test_groups_list)

# 确保有至少3个类别
unique_classes = np.unique(y_train)
if len(unique_classes) < 3:
    print(f"警告：训练数据中只有{len(unique_classes)}个类别，需要至少3个类别进行三分类分析")
    # 创建模拟的三分类标签（仅用于演示，实际使用时请替换为真实数据）
    print("创建模拟的三分类标签用于演示...")
    np.random.seed(SEED)
    y_train = pd.Series(np.random.choice(['1', '2', '3'], size=len(y_train)))
    unique_classes = np.unique(y_train)

# 确保测试集类别与训练集一致
test_unique = np.unique(y_test)
for cls in test_unique:
    if cls not in unique_classes:
        print(f"警告：测试集中发现训练集不存在的类别 {cls}，将其映射到最接近的类别")
        # 简单处理：将不存在的类别映射到第一个类别
        y_test.replace(cls, unique_classes[0], inplace=True)

# 重新编码目标变量，使其从 0 开始
class_mapping = {cls: idx for idx, cls in enumerate(unique_classes)}
y_train = y_train.map(class_mapping)
y_test = y_test.map(class_mapping)

# 处理类别不平衡 - 只对训练集应用SMOTE
smote = SMOTE(random_state=SEED)
X_train, y_train = smote.fit_resample(X_train, y_train)

# --------------------- 2. 特征筛选（仅在训练集内进行） ---------------------
# 2.1 单变量筛选 (Kruskal-Wallis)
significant_features = []
for feature in X_train.columns:
    groups = [X_train[y_train == group][feature] for group in y_train.unique()]
    try:
        _, p_val = kruskal(*groups)
    except ValueError:
        p_val = 1.0
    if p_val < 0.05:
        significant_features.append(feature)
X_train_filtered = X_train[significant_features]

# 2.2 去除高相关特征（相关系数>0.9）
corr_matrix = X_train_filtered.corr().abs()
upper = corr_matrix.where(np.triu(np.ones_like(corr_matrix), k=1).astype(bool))
to_drop = [col for col in upper.columns if any(upper[col] > 0.9)]
X_train_filtered = X_train_filtered.drop(columns=to_drop)

# 确保测试集也只保留筛选后的特征
# 处理测试集中可能不存在于训练集中的特征
test_features_to_keep = [f for f in X_train_filtered.columns if f in X_test.columns]
missing_features = [f for f in X_train_filtered.columns if f not in X_test.columns]
if missing_features:
    print(f"警告：测试集中缺少{len(missing_features)}个训练集特征，将使用默认值0填充")
    # 创建缺失特征并填充0
    for f in missing_features:
        X_test[f] = 0
    test_features_to_keep = X_train_filtered.columns

X_test_filtered = X_test[test_features_to_keep]

# --------------------- 3. 特征标准化 ---------------------
scaler = StandardScaler()

# 训练集标准化（转换为数组）
X_train_scaled_array = scaler.fit_transform(X_train_filtered)

# 测试集标准化（使用训练集的均值和方差）
X_test_scaled_array = scaler.transform(X_test_filtered)

# 将标准化后的数组转换回DataFrame（保留列名）
X_train_scaled = pd.DataFrame(
    X_train_scaled_array,
    columns=X_train_filtered.columns,
    index=X_train_filtered.index
)
X_test_scaled = pd.DataFrame(
    X_test_scaled_array,
    columns=X_test_filtered.columns,
    index=X_test_filtered.index
)

# --------------------- 4. 使用Lasso回归进行特征选择（交叉验证） ---------------------
# 创建 LassoCV 模型进行交叉验证
lasso_cv = LassoCV(cv=5, alphas=np.logspace(-4, 0, 100), n_jobs=-1)
lasso_cv.fit(X_train_scaled, y_train)

# 获取最优的 alpha 值
alpha_best = lasso_cv.alpha_
print(f"最优 alpha 值: {alpha_best}")

# 计算每个alpha对应的交叉验证误差和标准误差
mse_mean = lasso_cv.mse_path_.mean(axis=1)  # 平均MSE
mse_std = lasso_cv.mse_path_.std(axis=1)    # MSE的标准差

# 计算1-SE准则下的alpha
min_mse_idx = np.argmin(mse_mean)
min_mse = mse_mean[min_mse_idx]
mse_1se = min_mse + mse_std[min_mse_idx]
alpha_1se = np.max(lasso_cv.alphas_[mse_mean <= mse_1se])

print(f"1-SE alpha 值: {alpha_1se}")

# 创建不同正则化强度的 Lasso 模型
alphas = np.logspace(-4, 0, 100)
coefs = []

for alpha in alphas:
    lasso = Lasso(alpha=alpha)
    lasso.fit(X_train_scaled, y_train)
    coefs.append(lasso.coef_)

# 可视化系数随正则化强度的变化
plt.figure(figsize=(10, 6))
for i in range(X_train_scaled.shape[1]):
    plt.plot(alphas, [coef[i] for coef in coefs], marker='o', markersize=3, alpha=0.6)

plt.xscale('log')
plt.xlabel('Log lambda', fontsize=14)
plt.ylabel('Coefficients', fontsize=14)
plt.grid(True, which='both', linestyle='--', linewidth=0.5)
plt.legend([])
plt.savefig('./lasso_coef_path(LF-liver).tif', dpi=300)
plt.show()

# 进行特征筛选
lasso = Lasso(alpha=alpha_best)
lasso.fit(X_train_scaled, y_train)

# 获取非零系数对应的特征索引
selected_features_indices = np.nonzero(lasso.coef_)[0]

# 打印筛选出的特征名称
feature_names = X_train_scaled.columns
selected_feature_names = [feature_names[i] for i in selected_features_indices]
print(f"筛选出的特征数量: {len(selected_feature_names)}")
print("筛选出的特征:", selected_feature_names)

# 交叉验证曲线可视化
error_band_width = 0.1  # 可调整为任意正数，如1.5、2.0等

plt.figure(figsize=(10, 6))
plt.plot(lasso_cv.alphas_, mse_mean, color='red', linewidth=2, label='平均MSE')

# 方法1：使用垂直线段表示误差带（原方法）
for i in range(len(mse_mean)):
    plt.plot([lasso_cv.alphas_[i], lasso_cv.alphas_[i]], 
             [mse_mean[i] - error_band_width * mse_std[i], mse_mean[i] + error_band_width * mse_std[i]], 
             color='gray', linestyle='-', linewidth=0.5)

# 添加两条竖线
plt.axvline(x=alpha_best, color='gray', linestyle='--', label=f'最优 alpha: {alpha_best:.6f}')
plt.axvline(x=alpha_1se, color='gray', linestyle='--', label=f'1-SE alpha: {alpha_1se:.6f}')

# 更改y轴取值范围
#plt.ylim([0.19, 0.23])
plt.xscale('log')
plt.xlabel('Log (λ)', fontsize=14)
plt.ylabel('Mean Squared Error', fontsize=14)
plt.grid(True, which='both', linestyle='--', linewidth=0.5)

plt.savefig('./lasso_cv_curve(LF-liver).tif', dpi=300)
plt.show()

# 使用筛选后的特征
X_train_selected = X_train_scaled[selected_feature_names]
X_test_selected = X_test_scaled[selected_feature_names]  # 确保测试集使用相同特征

print(f"最终特征维度: {X_train_selected.shape}")

# --------------------- 使用SHAP库生成热力图 ---------------------
# 确保Lasso模型已正确拟合
lasso = Lasso(alpha=alpha_best)
lasso.fit(X_train_scaled, y_train)

# 验证系数存在
print("Lasso系数形状:", lasso.coef_.shape)

# 直接使用系数计算SHAP值
shap_values = X_train_scaled * lasso.coef_

# 筛选非零系数特征（仅用于可视化）
non_zero_features = X_train_scaled.columns[lasso.coef_ != 0]
non_zero_indices = [i for i, col in enumerate(X_train_scaled.columns) if col in non_zero_features]
non_zero_shap_values = shap_values.iloc[:, non_zero_indices]  # 使用DataFrame索引
non_zero_data = X_train_scaled[non_zero_features]

# 创建图形对象并指定为当前图形
fig = plt.figure(figsize=(12, 8))

# 生成SHAP图（关闭自动显示）
shap.summary_plot(non_zero_shap_values.values, non_zero_data, plot_type="dot", show=False)
#plt.title('SHAP Feature Importance (Lasso Regression)', fontsize=15)
plt.tight_layout()

# 保存图形并关闭
fig.savefig('./SHAP_Feature_Importance(LF-liver).tif', dpi=300, bbox_inches='tight')
plt.close(fig)  # 关闭图形以释放资源

# 定义交叉验证对象，确保每次分割一致
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)

# 随机森林模型参数选择
param_grid_rf = [
    {'n_estimators': [200, 300], 'max_depth': [15, 20], 'min_samples_split': [2, 3], 'min_samples_leaf': [1, 2]}
]
grid_search_rf = GridSearchCV(RandomForestClassifier(random_state=SEED), param_grid_rf, cv=cv)
grid_search_rf.fit(X_train_selected, y_train)
clf_rf = grid_search_rf.best_estimator_

# 决策树模型参数选择
param_grid_dt = [
    {'max_depth': [10, 15], 'min_samples_split': [2, 3], 'min_samples_leaf': [1, 2]}
]
grid_search_dt = GridSearchCV(DecisionTreeClassifier(random_state=SEED), param_grid_dt, cv=cv)
grid_search_dt.fit(X_train_selected, y_train)
clf_dt = grid_search_dt.best_estimator_

# 逻辑回归模型参数选择 - 确保支持多分类
param_grid_lg = [
    {'C': [0.1, 1, 10], 'penalty': ['l2', 'none']}
]
grid_search_lg = GridSearchCV(
    LogisticRegression(multi_class='multinomial', solver='lbfgs', random_state=SEED, max_iter=1000), 
    param_grid_lg, cv=cv
)
grid_search_lg.fit(X_train_selected, y_train)
clf_lg = grid_search_lg.best_estimator_

# SVM模型参数选择 - 确保支持多分类
param_grid_svm = [
    {'C': [1, 10, 100], 'kernel': ['linear', 'rbf'], 'gamma': ['scale', 'auto']}
]
grid_search_svm = GridSearchCV(SVC(probability=True, random_state=SEED, decision_function_shape='ovo'), param_grid_svm, cv=cv)
grid_search_svm.fit(X_train_selected, y_train)
clf_svm = grid_search_svm.best_estimator_

# GBM模型参数选择
param_grid_gbm = [
    {'n_estimators': [200, 300], 'learning_rate': [0.05, 0.1], 'max_depth': [5, 7]}
]
grid_search_gbm = GridSearchCV(GradientBoostingClassifier(random_state=SEED), param_grid_gbm, cv=cv)
grid_search_gbm.fit(X_train_selected, y_train)
clf_gbm = grid_search_gbm.best_estimator_

# XGBoost 模型参数选择 - 确保支持多分类
param_grid_xgb = {
    'n_estimators': [100, 200],
    'learning_rate': [0.05, 0.1],
    'max_depth': [3, 5],
    'objective': ['multi:softprob'],  # 多分类问题
    'num_class': [len(unique_classes)]  # 类别数量
}
grid_search_xgb = GridSearchCV(xgb.XGBClassifier(random_state=SEED), param_grid_xgb, cv=cv)
grid_search_xgb.fit(X_train_selected, y_train)
clf_xgb = grid_search_xgb.best_estimator_

# 创建堆叠分类器进行集成学习
estimators = [
    ('rf', clf_rf),
    ('dt', clf_dt),
    ('lg', clf_lg),
    ('svm', clf_svm),
    ('gbm', clf_gbm),
    ('xgb', clf_xgb)
]
stacking_clf = StackingClassifier(
    estimators=estimators, 
    final_estimator=LogisticRegression(random_state=SEED, max_iter=1000), 
    cv=cv
)
stacking_clf.fit(X_train_selected, y_train)

# 计算AUC的95%置信区间（使用DeLong方法）
def calculate_auc_ci(y_true, y_score, alpha=0.95):
    from scipy import stats
    from sklearn.metrics import roc_auc_score
    
    # 计算AUC
    auc = roc_auc_score(y_true, y_score)
    
    # 计算AUC的方差（DeLong方法）
    y_true = np.array(y_true)
    y_score = np.array(y_score)
    
    # 正例和反例的分数
    pos_scores = y_score[y_true == 1]
    neg_scores = y_score[y_true == 0]
    
    # 计算AUC的方差
    n1 = len(pos_scores)
    n2 = len(neg_scores)
    
    # 计算所有可能的正负样本对的比较结果
    tx = np.tile(pos_scores, (n2, 1))
    ty = np.tile(neg_scores, (n1, 1)).T
    
    # 计算AUC的方差
    p = np.mean(tx > ty) + 0.5 * np.mean(tx == ty)
    
    # 计算协方差矩阵
    pair_matrix = tx > ty
    pair_matrix_equal = tx == ty
    
    # 计算第一个方差分量
    variance_p1 = np.sum((np.mean(pair_matrix, axis=1) - p) ** 2) / (n1 - 1)
    
    # 计算第二个方差分量
    variance_p2 = np.sum((np.mean(pair_matrix, axis=0) - p) ** 2) / (n2 - 1)
    
    # 计算AUC的标准误差
    se_auc = np.sqrt((variance_p1 / n1) + (variance_p2 / n2))
    
    # 计算置信区间
    z = stats.norm.ppf(1 - (1 - alpha) / 2)
    lower = max(0, auc - z * se_auc)
    upper = min(1, auc + z * se_auc)
    
    return auc, lower, upper

# 计算二分类指标的95%置信区间（使用bootstrap方法）
def calculate_metrics_ci(y_true, y_pred, n_bootstraps=1000, alpha=0.95):
    from sklearn.utils import resample
    from sklearn.metrics import confusion_matrix
    
    # 存储每次bootstrap的结果
    sensitivity_bootstraps = []
    specificity_bootstraps = []
    ppv_bootstraps = []
    npv_bootstraps = []
    lr_pos_bootstraps = []
    lr_neg_bootstraps = []
    
    # 执行bootstrap
    for i in range(n_bootstraps):
        # 有放回地抽样
        indices = resample(range(len(y_true)))
        y_true_bs = y_true[indices]
        y_pred_bs = y_pred[indices]
        
        # 计算混淆矩阵
        cm = confusion_matrix(y_true_bs, y_pred_bs)
        
        if cm.shape == (1, 1):  # 处理只有一个类别的情况
            sensitivity = 1.0
            specificity = 1.0
            ppv = 1.0
            npv = 1.0
            lr_pos = float('inf')
            lr_neg = 0.0
        else:
            tn, fp, fn, tp = cm.ravel()
            
            sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
            specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
            ppv = tp / (tp + fp) if (tp + fp) > 0 else 0
            npv = tn / (tn + fn) if (tn + fn) > 0 else 0
            lr_pos = sensitivity / (1 - specificity) if (1 - specificity) > 0 else float('inf')
            lr_neg = (1 - sensitivity) / specificity if specificity > 0 else float('inf')
        
        # 存储结果
        sensitivity_bootstraps.append(sensitivity)
        specificity_bootstraps.append(specificity)
        ppv_bootstraps.append(ppv)
        npv_bootstraps.append(npv)
        lr_pos_bootstraps.append(lr_pos)
        lr_neg_bootstraps.append(lr_neg)
    
    # 计算置信区间
    def ci_interval(bootstraps):
        sorted_bootstraps = np.sort(bootstraps)
        lower = sorted_bootstraps[int((1 - alpha) / 2 * n_bootstraps)]
        upper = sorted_bootstraps[int((1 + alpha) / 2 * n_bootstraps)]
        return lower, upper
    
    sensitivity_ci = ci_interval(sensitivity_bootstraps)
    specificity_ci = ci_interval(specificity_bootstraps)
    ppv_ci = ci_interval(ppv_bootstraps)
    npv_ci = ci_interval(npv_bootstraps)
    lr_pos_ci = ci_interval(lr_pos_bootstraps)
    lr_neg_ci = ci_interval(lr_neg_bootstraps)
    
    return {
        'Sensitivity': (np.mean(sensitivity_bootstraps), sensitivity_ci),
        'Specificity': (np.mean(specificity_bootstraps), specificity_ci),
        'PPV': (np.mean(ppv_bootstraps), ppv_ci),
        'NPV': (np.mean(npv_bootstraps), npv_ci),
        'LR+': (np.mean(lr_pos_bootstraps), lr_pos_ci),
        'LR-': (np.mean(lr_neg_bootstraps), lr_neg_ci)
    }

# 计算模型的 ROC 曲线及 AUC 值的函数，增加置信区间计算
def calculate_roc_auc(model_name, y_pred_proba, y_test):
    unique_groups = np.unique(y_test)
    num_classes = len(unique_groups)
    fpr = {}
    tpr = {}
    roc_auc = {}
    roc_auc_ci = {}
    
    # 多分类情况
    for class_idx, group in enumerate(unique_groups):
        # 针对每个真实分组计算对应的ROC曲线和AUC值
        fpr[group], tpr[group], _ = roc_curve(y_test == group, y_pred_proba[:, class_idx])
        roc_auc[group], lower, upper = calculate_auc_ci(y_test == group, y_pred_proba[:, class_idx])
        roc_auc_ci[group] = (lower, upper)
    
    # 打印AUC值及其置信区间
    print(f"\n{model_name}模型的AUC值:")
    for group in unique_groups:
        print(f"类别 {group}: {roc_auc[group]:.4f} (95% CI: {roc_auc_ci[group][0]:.4f}-{roc_auc_ci[group][1]:.4f})")
    
    return fpr, tpr, roc_auc, roc_auc_ci  # 返回值增加roc_auc_ci

# 计算模型性能指标（敏感性、特异性、PPV、NPV、LR+、LR-）
def calculate_metrics(y_true, y_pred):
    from sklearn.metrics import confusion_matrix, classification_report
    
    # 获取唯一类别
    classes = np.unique(y_true)
    num_classes = len(classes)
    
    # 初始化结果字典
    metrics = {}
    
    # 计算每个类别的指标
    for i, cls in enumerate(classes):
        # 创建二分类标签：当前类别为正类，其他为负类
        y_true_binary = (y_true == cls).astype(int)
        y_pred_binary = (y_pred == cls).astype(int)
        
        # 计算混淆矩阵
        cm = confusion_matrix(y_true_binary, y_pred_binary)
        
        if cm.shape == (1, 1):  # 处理只有一个类别的情况
            if y_true_binary[0] == 1:
                # 所有样本都是正类
                tp = cm[0, 0]
                fn = 0
                fp = 0
                tn = 0
            else:
                # 所有样本都是负类
                tp = 0
                fn = 0
                fp = 0
                tn = cm[0, 0]
        else:
            tn, fp, fn, tp = cm.ravel()
        
        # 计算各项指标
        sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
        specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
        ppv = tp / (tp + fp) if (tp + fp) > 0 else 0
        npv = tn / (tn + fn) if (tn + fn) > 0 else 0
        lr_pos = sensitivity / (1 - specificity) if (1 - specificity) > 0 else float('inf')
        lr_neg = (1 - sensitivity) / specificity if specificity > 0 else float('inf')
        
        # 存储结果
        metrics[cls] = {
            'Sensitivity': sensitivity,
            'Specificity': specificity,
            'PPV': ppv,
            'NPV': npv,
            'LR+': lr_pos,
            'LR-': lr_neg
        }
    
    # 计算总体准确率
    accuracy = accuracy_score(y_true, y_pred)
    metrics['overall'] = {'Accuracy': accuracy}
    
    return metrics

# 获取各模型在测试集上的预测结果和预测概率（使用筛选后的特征）
y_pred_rf = clf_rf.predict(X_test_selected)
y_pred_proba_rf = clf_rf.predict_proba(X_test_selected)
y_pred_dt = clf_dt.predict(X_test_selected)
y_pred_proba_dt = clf_dt.predict_proba(X_test_selected)
y_pred_lg = clf_lg.predict(X_test_selected)
y_pred_proba_lg = clf_lg.predict_proba(X_test_selected)
y_pred_svm = clf_svm.predict(X_test_selected)
y_pred_proba_svm = clf_svm.predict_proba(X_test_selected)
y_pred_gbm = clf_gbm.predict(X_test_selected)
y_pred_proba_gbm = clf_gbm.predict_proba(X_test_selected)
y_pred_xgb = clf_xgb.predict(X_test_selected)
y_pred_proba_xgb = clf_xgb.predict_proba(X_test_selected)
y_pred_stacking = stacking_clf.predict(X_test_selected)
y_pred_proba_stacking = stacking_clf.predict_proba(X_test_selected)

# 获取各模型在训练集上的预测结果和预测概率
y_pred_train_rf = clf_rf.predict(X_train_selected)
y_pred_proba_train_rf = clf_rf.predict_proba(X_train_selected)
y_pred_train_dt = clf_dt.predict(X_train_selected)
y_pred_proba_train_dt = clf_dt.predict_proba(X_train_selected)
y_pred_train_lg = clf_lg.predict(X_train_selected)
y_pred_proba_train_lg = clf_lg.predict_proba(X_train_selected)
y_pred_train_svm = clf_svm.predict(X_train_selected)
y_pred_proba_train_svm = clf_svm.predict_proba(X_train_selected)
y_pred_train_gbm = clf_gbm.predict(X_train_selected)
y_pred_proba_train_gbm = clf_gbm.predict_proba(X_train_selected)
y_pred_train_xgb = clf_xgb.predict(X_train_selected)
y_pred_proba_train_xgb = clf_xgb.predict_proba(X_train_selected)
y_pred_train_stacking = stacking_clf.predict(X_train_selected)
y_pred_proba_train_stacking = stacking_clf.predict_proba(X_train_selected)

# 计算各模型的 ROC 曲线及 AUC 值 - 测试集
fpr_rf, tpr_rf, roc_auc_rf, roc_auc_ci_rf = calculate_roc_auc('RandomForest', y_pred_proba_rf, y_test)
fpr_dt, tpr_dt, roc_auc_dt, roc_auc_ci_dt = calculate_roc_auc('DecisionTree', y_pred_proba_dt, y_test)
fpr_lg, tpr_lg, roc_auc_lg, roc_auc_ci_lg = calculate_roc_auc('LogisticRegression', y_pred_proba_lg, y_test)
fpr_svm, tpr_svm, roc_auc_svm, roc_auc_ci_svm = calculate_roc_auc('SVM', y_pred_proba_svm, y_test)
fpr_gbm, tpr_gbm, roc_auc_gbm, roc_auc_ci_gbm = calculate_roc_auc('GBM', y_pred_proba_gbm, y_test)
fpr_xgb, tpr_xgb, roc_auc_xgb, roc_auc_ci_xgb = calculate_roc_auc('XGBoost', y_pred_proba_xgb, y_test)
fpr_stacking, tpr_stacking, roc_auc_stacking, roc_auc_ci_stacking = calculate_roc_auc('StackingClassifier', y_pred_proba_stacking, y_test)

# 计算各模型的 ROC 曲线及 AUC 值 - 训练集
fpr_train_rf, tpr_train_rf, roc_auc_train_rf, roc_auc_ci_train_rf = calculate_roc_auc('RandomForest', y_pred_proba_train_rf, y_train)
fpr_train_dt, tpr_train_dt, roc_auc_train_dt, roc_auc_ci_train_dt = calculate_roc_auc('DecisionTree', y_pred_proba_train_dt, y_train)
fpr_train_lg, tpr_train_lg, roc_auc_train_lg, roc_auc_ci_train_lg = calculate_roc_auc('LogisticRegression', y_pred_proba_train_lg, y_train)
fpr_train_svm, tpr_train_svm, roc_auc_train_svm, roc_auc_ci_train_svm = calculate_roc_auc('SVM', y_pred_proba_train_svm, y_train)
fpr_train_gbm, tpr_train_gbm, roc_auc_train_gbm, roc_auc_ci_train_gbm = calculate_roc_auc('GBM', y_pred_proba_train_gbm, y_train)
fpr_train_xgb, tpr_train_xgb, roc_auc_train_xgb, roc_auc_ci_train_xgb = calculate_roc_auc('XGBoost', y_pred_proba_train_xgb, y_train)
fpr_train_stacking, tpr_train_stacking, roc_auc_train_stacking, roc_auc_ci_train_stacking = calculate_roc_auc('StackingClassifier', y_pred_proba_train_stacking, y_train)

# 绘制测试集上所有类别的 ROC 曲线
plt.figure(figsize=(12, 8))
line_styles = ['-', '--', '-.', ':']
colors = ['r', 'g', 'b', 'c', 'm', 'y', 'k']
markers = ['o', 's', '^', '*', 'D', 'X', 'v']

# 获取唯一的类别列表
unique_classes = np.unique(y_test)
num_classes = len(unique_classes)

# 多分类问题，为每个类别绘制所有模型的曲线
for group_idx, group in enumerate(unique_classes):
    # 随机森林模型的曲线设置
    plt.plot(fpr_rf[group], tpr_rf[group], label=f'Random Forest -  {group}',
             lw=2, c=colors[0], marker=markers[0], linestyle=line_styles[group_idx % len(line_styles)])
    # 决策树模型的曲线设置
    plt.plot(fpr_dt[group], tpr_dt[group], label=f'Decision Tree -  {group}',
             lw=2, c=colors[1], marker=markers[1], linestyle=line_styles[group_idx % len(line_styles)])
    # 逻辑回归模型的曲线设置
    plt.plot(fpr_lg[group], tpr_lg[group], label=f'LogisticRegression -  {group}',
             lw=2, c=colors[2], marker=markers[2], linestyle=line_styles[group_idx % len(line_styles)])
    # SVM模型的曲线设置
    plt.plot(fpr_svm[group], tpr_svm[group], label=f'SVM -  {group}',
             lw=2, c=colors[3], marker=markers[3], linestyle=line_styles[group_idx % len(line_styles)])
    # GBM模型的曲线设置
    plt.plot(fpr_gbm[group], tpr_gbm[group], 
             label=f'GBM -  {group}',
             lw=2, c=colors[4], marker=markers[4], 
             linestyle=line_styles[group_idx % len(line_styles)])
    # XGBoost 模型的曲线设置
    plt.plot(fpr_xgb[group], tpr_xgb[group], 
             label=f'XGBoost -  {group}',
             lw=2, c=colors[5], marker=markers[5], 
             linestyle=line_styles[group_idx % len(line_styles)])
    # 堆叠分类器模型的曲线设置
    plt.plot(fpr_stacking[group], tpr_stacking[group], 
             label=f'StackingClassifier -  {group}',
             lw=2, c=colors[6], marker=markers[6], 
             linestyle=line_styles[group_idx % len(line_styles)])

plt.plot([0, 1], [0, 1], 'k--', lw=2)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('1-Specificity')
plt.ylabel('Sensitivity')
#plt.title('ROC Curves for All Classes (Test Set)')
plt.legend(loc="lower right", fontsize='small')
plt.grid(True, which='both', linestyle='--', linewidth=0.5)
plt.savefig('./LF-LML-roc(test).tif', dpi=300)
plt.show()

# 绘制训练集上所有类别的 ROC 曲线
plt.figure(figsize=(12, 8))
line_styles = ['-', '--', '-.', ':']
colors = ['r', 'g', 'b', 'c', 'm', 'y', 'k']
markers = ['o', 's', '^', '*', 'D', 'X', 'v']

# 获取唯一的类别列表
unique_classes = np.unique(y_train)
num_classes = len(unique_classes)

# 多分类问题，为每个类别绘制所有模型的曲线
for group_idx, group in enumerate(unique_classes):
    # 随机森林模型的曲线设置
    plt.plot(fpr_train_rf[group], tpr_train_rf[group], label=f'Random Forest -  {group}',
             lw=2, c=colors[0], marker=markers[0], linestyle=line_styles[group_idx % len(line_styles)])
    # 决策树模型的曲线设置
    plt.plot(fpr_train_dt[group], tpr_train_dt[group], label=f'Decision Tree -  {group}',
             lw=2, c=colors[1], marker=markers[1], linestyle=line_styles[group_idx % len(line_styles)])
    # 逻辑回归模型的曲线设置
    plt.plot(fpr_train_lg[group], tpr_train_lg[group], label=f'LogisticRegression -  {group}',
             lw=2, c=colors[2], marker=markers[2], linestyle=line_styles[group_idx % len(line_styles)])
    # SVM模型的曲线设置
    plt.plot(fpr_train_svm[group], tpr_train_svm[group], label=f'SVM -  {group}',
             lw=2, c=colors[3], marker=markers[3], linestyle=line_styles[group_idx % len(line_styles)])
    # GBM模型的曲线设置
    plt.plot(fpr_train_gbm[group], tpr_train_gbm[group], 
             label=f'GBM -  {group}',
             lw=2, c=colors[4], marker=markers[4], 
             linestyle=line_styles[group_idx % len(line_styles)])
    # XGBoost 模型的曲线设置
    plt.plot(fpr_train_xgb[group], tpr_train_xgb[group], 
             label=f'XGBoost -  {group}',
             lw=2, c=colors[5], marker=markers[5], 
             linestyle=line_styles[group_idx % len(line_styles)])
    # 堆叠分类器模型的曲线设置
    plt.plot(fpr_train_stacking[group], tpr_train_stacking[group], 
             label=f'StackingClassifier -  {group}',
             lw=2, c=colors[6], marker=markers[6], 
             linestyle=line_styles[group_idx % len(line_styles)])

plt.plot([0, 1], [0, 1], 'k--', lw=2)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('1-Specificity')
plt.ylabel('Sensitivity')
#plt.title('ROC Curves for All Classes (Training Set)')
plt.legend(loc="lower right", fontsize='small')
plt.grid(True, which='both', linestyle='--', linewidth=0.5)
plt.savefig('./LF-LML-roc(train).tif', dpi=300)
plt.show()

# 计算并打印各模型的性能指标
def print_model_metrics(model_name, y_true, y_pred, dataset_type):
    metrics = calculate_metrics(y_true, y_pred)
    
    print(f"\n{model_name} Model ({dataset_type}):")
    
    # 获取所有类别键（排除'overall'）
    class_keys = [k for k in metrics.keys() if k != 'overall']
    
    # 尝试对类别键进行排序
    try:
        # 尝试数值排序
        class_keys = sorted(class_keys, key=lambda x: int(x))
    except (ValueError, TypeError):
        # 如果无法数值排序，则进行字符串排序
        class_keys = sorted(class_keys, key=lambda x: str(x))
    
    # 打印每个类别的指标
    for cls in class_keys:
        print(f"\n  类别 {cls}:")
        print(f"    Sensitivity: {metrics[cls]['Sensitivity']:.4f}")
        print(f"    Specificity: {metrics[cls]['Specificity']:.4f}")
        print(f"    Positive Predictive Value (PPV): {metrics[cls]['PPV']:.4f}")
        print(f"    Negative Predictive Value (NPV): {metrics[cls]['NPV']:.4f}")
        print(f"    Positive Likelihood Ratio (LR+): {metrics[cls]['LR+']:.4f}")
        print(f"    Negative Likelihood Ratio (LR-): {metrics[cls]['LR-']:.4f}")
    
    # 打印总体准确率
    print(f"\n  总体准确率: {metrics['overall']['Accuracy']:.4f}")

# 打印测试集指标
print_model_metrics("Random Forest", y_test, y_pred_rf, "Test Set")
print_model_metrics("Decision Tree", y_test, y_pred_dt, "Test Set")
print_model_metrics("Logistic Regression", y_test, y_pred_lg, "Test Set")
print_model_metrics("SVM", y_test, y_pred_svm, "Test Set")
print_model_metrics("Gradient Boosting", y_test, y_pred_gbm, "Test Set")
print_model_metrics("XGBoost", y_test, y_pred_xgb, "Test Set")
print_model_metrics("Stacking Classifier", y_test, y_pred_stacking, "Test Set")

# 打印训练集指标
print_model_metrics("Random Forest", y_train, y_pred_train_rf, "Training Set")
print_model_metrics("Decision Tree", y_train, y_pred_train_dt, "Training Set")
print_model_metrics("Logistic Regression", y_train, y_pred_train_lg, "Training Set")
print_model_metrics("SVM", y_train, y_pred_train_svm, "Training Set")
print_model_metrics("Gradient Boosting", y_train, y_pred_train_gbm, "Training Set")
print_model_metrics("XGBoost", y_train, y_pred_train_xgb, "Training Set")
print_model_metrics("Stacking Classifier", y_train, y_pred_train_stacking, "Training Set")

# 保存所有模型的测试集结果，按分类保存
for model_name, y_pred_proba, y_pred in zip(
    ['RandomForest', 'DecisionTree', 'LogisticRegression', 'SVM', 'GBM', 'XGBoost', 'StackingClassifier'],
    [y_pred_proba_rf, y_pred_proba_dt, y_pred_proba_lg, y_pred_proba_svm, y_pred_proba_gbm, y_pred_proba_xgb, y_pred_proba_stacking],
    [y_pred_rf, y_pred_dt, y_pred_lg, y_pred_svm, y_pred_gbm, y_pred_xgb, y_pred_stacking]
):
    # 创建模型目录
    model_dir = os.path.join(results_dir, "test", model_name)
    os.makedirs(model_dir, exist_ok=True)
    
    # 保存原始预测结果
    custom_name = f'LF-LML-{model_name}(test)'
    np.save(os.path.join(model_dir, f'{custom_name}_y_true.npy'), y_test)
    np.save(os.path.join(model_dir, f'{custom_name}_y_proba.npy'), y_pred_proba)
    np.save(os.path.join(model_dir, f'{custom_name}_y_pred.npy'), y_pred)
    print(f"已保存 {custom_name} 模型的测试集原始结果")
    
    # 按分类保存预测结果
    for class_idx, class_name in enumerate(unique_classes):
        class_dir = os.path.join(model_dir, f"class_{class_name}")
        os.makedirs(class_dir, exist_ok=True)
        
        # 为当前类别创建二分类标签
        y_true_binary = (y_test == class_name).astype(int)
        y_pred_binary = (y_pred == class_name).astype(int)
        y_proba_binary = y_pred_proba[:, class_idx]
        
        # 保存二分类结果
        np.save(os.path.join(class_dir, f'{custom_name}_class{class_name}_y_true.npy'), y_true_binary)
        np.save(os.path.join(class_dir, f'{custom_name}_class{class_name}_y_proba.npy'), y_proba_binary)
        np.save(os.path.join(class_dir, f'{custom_name}_class{class_name}_y_pred.npy'), y_pred_binary)
        print(f"已保存 {custom_name} 模型类别 {class_name} 的测试集二分类结果")

# 保存所有模型的训练集结果，按分类保存
for model_name, y_pred_proba, y_pred in zip(
    ['RandomForest', 'DecisionTree', 'LogisticRegression', 'SVM', 'GBM', 'XGBoost', 'StackingClassifier'],
    [y_pred_proba_train_rf, y_pred_proba_train_dt, y_pred_proba_train_lg, y_pred_proba_train_svm, y_pred_proba_train_gbm, y_pred_proba_train_xgb, y_pred_proba_train_stacking],
    [y_pred_train_rf, y_pred_train_dt, y_pred_train_lg, y_pred_train_svm, y_pred_train_gbm, y_pred_train_xgb, y_pred_train_stacking]
):
    # 创建模型目录
    model_dir = os.path.join(results_dir, "train", model_name)
    os.makedirs(model_dir, exist_ok=True)
    
    # 保存原始预测结果
    custom_name = f'LF-LML-{model_name}(train)'
    np.save(os.path.join(model_dir, f'{custom_name}_y_true.npy'), y_train)
    np.save(os.path.join(model_dir, f'{custom_name}_y_proba.npy'), y_pred_proba)
    np.save(os.path.join(model_dir, f'{custom_name}_y_pred.npy'), y_pred)
    print(f"已保存 {custom_name} 模型的训练集原始结果")
    
    # 按分类保存预测结果
    for class_idx, class_name in enumerate(unique_classes):
        class_dir = os.path.join(model_dir, f"class_{class_name}")
        os.makedirs(class_dir, exist_ok=True)
        
        # 为当前类别创建二分类标签
        y_true_binary = (y_train == class_name).astype(int)
        y_pred_binary = (y_pred == class_name).astype(int)
        y_proba_binary = y_pred_proba[:, class_idx]
        
        # 保存二分类结果
        np.save(os.path.join(class_dir, f'{custom_name}_class{class_name}_y_true.npy'), y_true_binary)
        np.save(os.path.join(class_dir, f'{custom_name}_class{class_name}_y_proba.npy'), y_proba_binary)
        np.save(os.path.join(class_dir, f'{custom_name}_class{class_name}_y_pred.npy'), y_pred_binary)
        print(f"已保存 {custom_name} 模型类别 {class_name} 的训练集二分类结果")


In [ ]:
import os
import numpy as np
import pandas as pd
import SimpleITK as sitk
from radiomics import featureextractor
import glob
from scipy.stats import kruskal
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LassoCV, Lasso, LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, StackingClassifier
from sklearn.metrics import (roc_auc_score, classification_report,
                             confusion_matrix, ConfusionMatrixDisplay,
                             accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc)
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.calibration import calibration_curve
import matplotlib.pyplot as plt
import xgboost as xgb
from imblearn.over_sampling import SMOTE  # 确保已安装imblearn库
from sklearn.model_selection import cross_val_score
import shap

# 设置随机种子，确保结果可复现
def seed_everything(seed=42):
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    # 如果使用PyTorch，也需要设置相关种子
    # torch.manual_seed(seed)
    # torch.cuda.manual_seed_all(seed)
    return seed

SEED = seed_everything(42)

# 定义训练集和测试集的图像和掩膜文件的文件夹路径
# 训练集路径
train_image_folder = "C:/Users/YK/Desktop/train_spleen"
train_mask_folder = "C:/Users/YK/Desktop/train_spleen-mask"

# 测试集路径
test_image_folder = "C:/Users/YK/Desktop/test_spleen"
test_mask_folder = "C:/Users/YK/Desktop/test_spleen-mask"

# 创建结果保存目录
results_dir = "LF-LML(E)"
os.makedirs(results_dir, exist_ok=True)

# 初始化特征提取器
extractor = featureextractor.RadiomicsFeatureExtractor()

# 启用所有标准特征类
feature_classes = [
    'firstorder', 'glcm', 'glrlm', 'glszm',
    'ngtdm', 'gldm', 'shape2D'  # 使用shape2D替代shape
]
for fc in feature_classes:
    extractor.enableFeatureClassByName(fc)

# 启用所有可用的图像类型
image_types = [
    'Wavelet', 'LoG', 'Square', 'SquareRoot',
    'Logarithm', 'Exponential', 'Gradient'
]
for it in image_types:
    extractor.enableImageTypeByName(it)

# 配置额外参数以增加特征
extractor.settings.update({
    'binWidth': 25,
    'normalize': True,
    'force2D': True  # 确保处理为2D图像
})

# 定义函数用于提取数据集特征
def extract_dataset_features(image_folder, mask_folder):
    features_list = []
    groups_list = []
    
    # 遍历文件夹中的文件
    for filename in os.listdir(image_folder):
        if filename.endswith('.jpg'):
            base_name = os.path.splitext(filename)[0]
            group_info = base_name.split('_')[0].split('F')[1]
            image_path = os.path.join(image_folder, filename)
            
            # 使用通配符匹配所有可能的掩膜文件
            mask_pattern = os.path.join(mask_folder, base_name + ".nrrd")
            mask_files = glob.glob(mask_pattern)
            
            for mask_path in mask_files:
                try:
                    # 读取图像和掩膜
                    image = sitk.ReadImage(image_path)
                    mask = sitk.ReadImage(mask_path)
                    
                    # 维度处理
                    if image.GetDimension() != mask.GetDimension():
                        if image.GetDimension() == 2 and mask.GetDimension() == 3:
                            mask = mask[:, :, 0]  # 取第一个切片
                        else:
                            print(f"维度不匹配已跳过：{image_path} | {mask_path}")
                            continue
                            
                    # 图像预处理
                    if image.GetNumberOfComponentsPerPixel() > 1:
                        image = sitk.VectorIndexSelectionCast(image, 0, sitk.sitkUInt8)
                    
                    # 特征提取
                    features = extractor.execute(image, mask)
                    
                    # 过滤诊断信息并保留所有特征
                    feature_values = {
                        k: v for k, v in features.items()
                        if not k.startswith('diagnostics')
                    }
                    
                    features_list.append(feature_values)
                    groups_list.append(group_info)
                    
                except Exception as e:
                    print(f"处理失败 {mask_path}: {str(e)}")
                    continue
    
    return features_list, groups_list

# 提取训练集特征
print("正在提取训练集特征...")
train_features_list, train_groups_list = extract_dataset_features(train_image_folder, train_mask_folder)

# 提取测试集特征
print("正在提取测试集特征...")
test_features_list, test_groups_list = extract_dataset_features(test_image_folder, test_mask_folder)

# 将特征转换为 DataFrame
X_train = pd.DataFrame(train_features_list)
X_test = pd.DataFrame(test_features_list)

# 将分组信息转换为 Series
y_train = pd.Series(train_groups_list)
y_test = pd.Series(test_groups_list)

# 确保有至少3个类别
unique_classes = np.unique(y_train)
if len(unique_classes) < 3:
    print(f"警告：训练数据中只有{len(unique_classes)}个类别，需要至少3个类别进行三分类分析")
    # 创建模拟的三分类标签（仅用于演示，实际使用时请替换为真实数据）
    print("创建模拟的三分类标签用于演示...")
    np.random.seed(SEED)
    y_train = pd.Series(np.random.choice(['1', '2', '3'], size=len(y_train)))
    unique_classes = np.unique(y_train)

# 确保测试集类别与训练集一致
test_unique = np.unique(y_test)
for cls in test_unique:
    if cls not in unique_classes:
        print(f"警告：测试集中发现训练集不存在的类别 {cls}，将其映射到最接近的类别")
        # 简单处理：将不存在的类别映射到第一个类别
        y_test.replace(cls, unique_classes[0], inplace=True)

# 重新编码目标变量，使其从 0 开始
class_mapping = {cls: idx for idx, cls in enumerate(unique_classes)}
y_train = y_train.map(class_mapping)
y_test = y_test.map(class_mapping)

# 处理类别不平衡 - 只对训练集应用SMOTE
smote = SMOTE(random_state=SEED)
X_train, y_train = smote.fit_resample(X_train, y_train)

# --------------------- 2. 特征筛选（仅在训练集内进行） ---------------------
# 2.1 单变量筛选 (Kruskal-Wallis)
significant_features = []
for feature in X_train.columns:
    groups = [X_train[y_train == group][feature] for group in y_train.unique()]
    try:
        _, p_val = kruskal(*groups)
    except ValueError:
        p_val = 1.0
    if p_val < 0.05:
        significant_features.append(feature)
X_train_filtered = X_train[significant_features]

# 2.2 去除高相关特征（相关系数>0.9）
corr_matrix = X_train_filtered.corr().abs()
upper = corr_matrix.where(np.triu(np.ones_like(corr_matrix), k=1).astype(bool))
to_drop = [col for col in upper.columns if any(upper[col] > 0.9)]
X_train_filtered = X_train_filtered.drop(columns=to_drop)

# 确保测试集也只保留筛选后的特征
# 处理测试集中可能不存在于训练集中的特征
test_features_to_keep = [f for f in X_train_filtered.columns if f in X_test.columns]
missing_features = [f for f in X_train_filtered.columns if f not in X_test.columns]
if missing_features:
    print(f"警告：测试集中缺少{len(missing_features)}个训练集特征，将使用默认值0填充")
    # 创建缺失特征并填充0
    for f in missing_features:
        X_test[f] = 0
    test_features_to_keep = X_train_filtered.columns

X_test_filtered = X_test[test_features_to_keep]

# --------------------- 3. 特征标准化 ---------------------
scaler = StandardScaler()

# 训练集标准化（转换为数组）
X_train_scaled_array = scaler.fit_transform(X_train_filtered)

# 测试集标准化（使用训练集的均值和方差）
X_test_scaled_array = scaler.transform(X_test_filtered)

# 将标准化后的数组转换回DataFrame（保留列名）
X_train_scaled = pd.DataFrame(
    X_train_scaled_array,
    columns=X_train_filtered.columns,
    index=X_train_filtered.index
)
X_test_scaled = pd.DataFrame(
    X_test_scaled_array,
    columns=X_test_filtered.columns,
    index=X_test_filtered.index
)

# --------------------- 4. 使用Lasso回归进行特征选择（交叉验证） ---------------------
# 创建 LassoCV 模型进行交叉验证
lasso_cv = LassoCV(cv=5, alphas=np.logspace(-4, 0, 100), n_jobs=-1)
lasso_cv.fit(X_train_scaled, y_train)

# 获取最优的 alpha 值
alpha_best = lasso_cv.alpha_
print(f"最优 alpha 值: {alpha_best}")

# 计算每个alpha对应的交叉验证误差和标准误差
mse_mean = lasso_cv.mse_path_.mean(axis=1)  # 平均MSE
mse_std = lasso_cv.mse_path_.std(axis=1)    # MSE的标准差

# 计算1-SE准则下的alpha
min_mse_idx = np.argmin(mse_mean)
min_mse = mse_mean[min_mse_idx]
mse_1se = min_mse + mse_std[min_mse_idx]
alpha_1se = np.max(lasso_cv.alphas_[mse_mean <= mse_1se])

print(f"1-SE alpha 值: {alpha_1se}")

# 创建不同正则化强度的 Lasso 模型
alphas = np.logspace(-4, 0, 100)
coefs = []

for alpha in alphas:
    lasso = Lasso(alpha=alpha)
    lasso.fit(X_train_scaled, y_train)
    coefs.append(lasso.coef_)

# 可视化系数随正则化强度的变化
plt.figure(figsize=(10, 6))
for i in range(X_train_scaled.shape[1]):
    plt.plot(alphas, [coef[i] for coef in coefs], marker='o', markersize=3, alpha=0.6)

plt.xscale('log')
plt.xlabel('Log lambda', fontsize=14)
plt.ylabel('Coefficients', fontsize=14)
plt.grid(True, which='both', linestyle='--', linewidth=0.5)
plt.legend([])
plt.savefig('./lasso_coef_path(LF-spleen).tif', dpi=300)
plt.show()

# 进行特征筛选
lasso = Lasso(alpha=alpha_best)
lasso.fit(X_train_scaled, y_train)

# 获取非零系数对应的特征索引
selected_features_indices = np.nonzero(lasso.coef_)[0]

# 打印筛选出的特征名称
feature_names = X_train_scaled.columns
selected_feature_names = [feature_names[i] for i in selected_features_indices]
print(f"筛选出的特征数量: {len(selected_feature_names)}")
print("筛选出的特征:", selected_feature_names)

# 交叉验证曲线可视化
error_band_width = 0.1  # 可调整为任意正数，如1.5、2.0等

plt.figure(figsize=(10, 6))
plt.plot(lasso_cv.alphas_, mse_mean, color='red', linewidth=2, label='平均MSE')

# 方法1：使用垂直线段表示误差带（原方法）
for i in range(len(mse_mean)):
    plt.plot([lasso_cv.alphas_[i], lasso_cv.alphas_[i]], 
             [mse_mean[i] - error_band_width * mse_std[i], mse_mean[i] + error_band_width * mse_std[i]], 
             color='gray', linestyle='-', linewidth=0.5)

# 添加两条竖线
plt.axvline(x=alpha_best, color='gray', linestyle='--', label=f'最优 alpha: {alpha_best:.6f}')
plt.axvline(x=alpha_1se, color='gray', linestyle='--', label=f'1-SE alpha: {alpha_1se:.6f}')

# 更改y轴取值范围
#plt.ylim([0.19, 0.23])
plt.xscale('log')
plt.xlabel('Log (λ)', fontsize=14)
plt.ylabel('Mean Squared Error', fontsize=14)
plt.grid(True, which='both', linestyle='--', linewidth=0.5)

plt.savefig('./lasso_cv_curve(LF-spleen).tif', dpi=300)
plt.show()

# 使用筛选后的特征
X_train_selected = X_train_scaled[selected_feature_names]
X_test_selected = X_test_scaled[selected_feature_names]  # 确保测试集使用相同特征

print(f"最终特征维度: {X_train_selected.shape}")

# --------------------- 使用SHAP库生成热力图 ---------------------
# 确保Lasso模型已正确拟合
lasso = Lasso(alpha=alpha_best)
lasso.fit(X_train_scaled, y_train)

# 验证系数存在
print("Lasso系数形状:", lasso.coef_.shape)

# 直接使用系数计算SHAP值
shap_values = X_train_scaled * lasso.coef_

# 筛选非零系数特征（仅用于可视化）
non_zero_features = X_train_scaled.columns[lasso.coef_ != 0]
non_zero_indices = [i for i, col in enumerate(X_train_scaled.columns) if col in non_zero_features]
non_zero_shap_values = shap_values.iloc[:, non_zero_indices]  # 使用DataFrame索引
non_zero_data = X_train_scaled[non_zero_features]

# 创建图形对象并指定为当前图形
fig = plt.figure(figsize=(12, 8))

# 生成SHAP图（关闭自动显示）
shap.summary_plot(non_zero_shap_values.values, non_zero_data, plot_type="dot", show=False)
#plt.title('SHAP Feature Importance (Lasso Regression)', fontsize=15)
plt.tight_layout()

# 保存图形并关闭
fig.savefig('./SHAP_Feature_Importance(LF-spleen).tif', dpi=300, bbox_inches='tight')
plt.close(fig)  # 关闭图形以释放资源

# 定义交叉验证对象，确保每次分割一致
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)

# 随机森林模型参数选择
param_grid_rf = [
    {'n_estimators': [200, 300], 'max_depth': [15, 20], 'min_samples_split': [2, 3], 'min_samples_leaf': [1, 2]}
]
grid_search_rf = GridSearchCV(RandomForestClassifier(random_state=SEED), param_grid_rf, cv=cv)
grid_search_rf.fit(X_train_selected, y_train)
clf_rf = grid_search_rf.best_estimator_

# 决策树模型参数选择
param_grid_dt = [
    {'max_depth': [10, 15], 'min_samples_split': [2, 3], 'min_samples_leaf': [1, 2]}
]
grid_search_dt = GridSearchCV(DecisionTreeClassifier(random_state=SEED), param_grid_dt, cv=cv)
grid_search_dt.fit(X_train_selected, y_train)
clf_dt = grid_search_dt.best_estimator_

# 逻辑回归模型参数选择 - 确保支持多分类
param_grid_lg = [
    {'C': [0.1, 1, 10], 'penalty': ['l2', 'none']}
]
grid_search_lg = GridSearchCV(
    LogisticRegression(multi_class='multinomial', solver='lbfgs', random_state=SEED, max_iter=1000), 
    param_grid_lg, cv=cv
)
grid_search_lg.fit(X_train_selected, y_train)
clf_lg = grid_search_lg.best_estimator_

# SVM模型参数选择 - 确保支持多分类
param_grid_svm = [
    {'C': [1, 10, 100], 'kernel': ['linear', 'rbf'], 'gamma': ['scale', 'auto']}
]
grid_search_svm = GridSearchCV(SVC(probability=True, random_state=SEED, decision_function_shape='ovo'), param_grid_svm, cv=cv)
grid_search_svm.fit(X_train_selected, y_train)
clf_svm = grid_search_svm.best_estimator_

# GBM模型参数选择
param_grid_gbm = [
    {'n_estimators': [200, 300], 'learning_rate': [0.05, 0.1], 'max_depth': [5, 7]}
]
grid_search_gbm = GridSearchCV(GradientBoostingClassifier(random_state=SEED), param_grid_gbm, cv=cv)
grid_search_gbm.fit(X_train_selected, y_train)
clf_gbm = grid_search_gbm.best_estimator_

# XGBoost 模型参数选择 - 确保支持多分类
param_grid_xgb = {
    'n_estimators': [100, 200],
    'learning_rate': [0.05, 0.1],
    'max_depth': [3, 5],
    'objective': ['multi:softprob'],  # 多分类问题
    'num_class': [len(unique_classes)]  # 类别数量
}
grid_search_xgb = GridSearchCV(xgb.XGBClassifier(random_state=SEED), param_grid_xgb, cv=cv)
grid_search_xgb.fit(X_train_selected, y_train)
clf_xgb = grid_search_xgb.best_estimator_

# 创建堆叠分类器进行集成学习
estimators = [
    ('rf', clf_rf),
    ('dt', clf_dt),
    ('lg', clf_lg),
    ('svm', clf_svm),
    ('gbm', clf_gbm),
    ('xgb', clf_xgb)
]
stacking_clf = StackingClassifier(
    estimators=estimators, 
    final_estimator=LogisticRegression(random_state=SEED, max_iter=1000), 
    cv=cv
)
stacking_clf.fit(X_train_selected, y_train)

# 计算AUC的95%置信区间（使用DeLong方法）
def calculate_auc_ci(y_true, y_score, alpha=0.95):
    from scipy import stats
    from sklearn.metrics import roc_auc_score
    
    # 计算AUC
    auc = roc_auc_score(y_true, y_score)
    
    # 计算AUC的方差（DeLong方法）
    y_true = np.array(y_true)
    y_score = np.array(y_score)
    
    # 正例和反例的分数
    pos_scores = y_score[y_true == 1]
    neg_scores = y_score[y_true == 0]
    
    # 计算AUC的方差
    n1 = len(pos_scores)
    n2 = len(neg_scores)
    
    # 计算所有可能的正负样本对的比较结果
    tx = np.tile(pos_scores, (n2, 1))
    ty = np.tile(neg_scores, (n1, 1)).T
    
    # 计算AUC的方差
    p = np.mean(tx > ty) + 0.5 * np.mean(tx == ty)
    
    # 计算协方差矩阵
    pair_matrix = tx > ty
    pair_matrix_equal = tx == ty
    
    # 计算第一个方差分量
    variance_p1 = np.sum((np.mean(pair_matrix, axis=1) - p) ** 2) / (n1 - 1)
    
    # 计算第二个方差分量
    variance_p2 = np.sum((np.mean(pair_matrix, axis=0) - p) ** 2) / (n2 - 1)
    
    # 计算AUC的标准误差
    se_auc = np.sqrt((variance_p1 / n1) + (variance_p2 / n2))
    
    # 计算置信区间
    z = stats.norm.ppf(1 - (1 - alpha) / 2)
    lower = max(0, auc - z * se_auc)
    upper = min(1, auc + z * se_auc)
    
    return auc, lower, upper

# 计算二分类指标的95%置信区间（使用bootstrap方法）
def calculate_metrics_ci(y_true, y_pred, n_bootstraps=1000, alpha=0.95):
    from sklearn.utils import resample
    from sklearn.metrics import confusion_matrix
    
    # 存储每次bootstrap的结果
    sensitivity_bootstraps = []
    specificity_bootstraps = []
    ppv_bootstraps = []
    npv_bootstraps = []
    lr_pos_bootstraps = []
    lr_neg_bootstraps = []
    
    # 执行bootstrap
    for i in range(n_bootstraps):
        # 有放回地抽样
        indices = resample(range(len(y_true)))
        y_true_bs = y_true[indices]
        y_pred_bs = y_pred[indices]
        
        # 计算混淆矩阵
        cm = confusion_matrix(y_true_bs, y_pred_bs)
        
        if cm.shape == (1, 1):  # 处理只有一个类别的情况
            sensitivity = 1.0
            specificity = 1.0
            ppv = 1.0
            npv = 1.0
            lr_pos = float('inf')
            lr_neg = 0.0
        else:
            tn, fp, fn, tp = cm.ravel()
            
            sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
            specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
            ppv = tp / (tp + fp) if (tp + fp) > 0 else 0
            npv = tn / (tn + fn) if (tn + fn) > 0 else 0
            lr_pos = sensitivity / (1 - specificity) if (1 - specificity) > 0 else float('inf')
            lr_neg = (1 - sensitivity) / specificity if specificity > 0 else float('inf')
        
        # 存储结果
        sensitivity_bootstraps.append(sensitivity)
        specificity_bootstraps.append(specificity)
        ppv_bootstraps.append(ppv)
        npv_bootstraps.append(npv)
        lr_pos_bootstraps.append(lr_pos)
        lr_neg_bootstraps.append(lr_neg)
    
    # 计算置信区间
    def ci_interval(bootstraps):
        sorted_bootstraps = np.sort(bootstraps)
        lower = sorted_bootstraps[int((1 - alpha) / 2 * n_bootstraps)]
        upper = sorted_bootstraps[int((1 + alpha) / 2 * n_bootstraps)]
        return lower, upper
    
    sensitivity_ci = ci_interval(sensitivity_bootstraps)
    specificity_ci = ci_interval(specificity_bootstraps)
    ppv_ci = ci_interval(ppv_bootstraps)
    npv_ci = ci_interval(npv_bootstraps)
    lr_pos_ci = ci_interval(lr_pos_bootstraps)
    lr_neg_ci = ci_interval(lr_neg_bootstraps)
    
    return {
        'Sensitivity': (np.mean(sensitivity_bootstraps), sensitivity_ci),
        'Specificity': (np.mean(specificity_bootstraps), specificity_ci),
        'PPV': (np.mean(ppv_bootstraps), ppv_ci),
        'NPV': (np.mean(npv_bootstraps), npv_ci),
        'LR+': (np.mean(lr_pos_bootstraps), lr_pos_ci),
        'LR-': (np.mean(lr_neg_bootstraps), lr_neg_ci)
    }

# 计算模型的 ROC 曲线及 AUC 值的函数，增加置信区间计算
def calculate_roc_auc(model_name, y_pred_proba, y_test):
    unique_groups = np.unique(y_test)
    num_classes = len(unique_groups)
    fpr = {}
    tpr = {}
    roc_auc = {}
    roc_auc_ci = {}
    
    # 多分类情况
    for class_idx, group in enumerate(unique_groups):
        # 针对每个真实分组计算对应的ROC曲线和AUC值
        fpr[group], tpr[group], _ = roc_curve(y_test == group, y_pred_proba[:, class_idx])
        roc_auc[group], lower, upper = calculate_auc_ci(y_test == group, y_pred_proba[:, class_idx])
        roc_auc_ci[group] = (lower, upper)
    
    # 打印AUC值及其置信区间
    print(f"\n{model_name}模型的AUC值:")
    for group in unique_groups:
        print(f"类别 {group}: {roc_auc[group]:.4f} (95% CI: {roc_auc_ci[group][0]:.4f}-{roc_auc_ci[group][1]:.4f})")
    
    return fpr, tpr, roc_auc, roc_auc_ci  # 返回值增加roc_auc_ci

# 计算模型性能指标（敏感性、特异性、PPV、NPV、LR+、LR-）
def calculate_metrics(y_true, y_pred):
    from sklearn.metrics import confusion_matrix, classification_report
    
    # 获取唯一类别
    classes = np.unique(y_true)
    num_classes = len(classes)
    
    # 初始化结果字典
    metrics = {}
    
    # 计算每个类别的指标
    for i, cls in enumerate(classes):
        # 创建二分类标签：当前类别为正类，其他为负类
        y_true_binary = (y_true == cls).astype(int)
        y_pred_binary = (y_pred == cls).astype(int)
        
        # 计算混淆矩阵
        cm = confusion_matrix(y_true_binary, y_pred_binary)
        
        if cm.shape == (1, 1):  # 处理只有一个类别的情况
            if y_true_binary[0] == 1:
                # 所有样本都是正类
                tp = cm[0, 0]
                fn = 0
                fp = 0
                tn = 0
            else:
                # 所有样本都是负类
                tp = 0
                fn = 0
                fp = 0
                tn = cm[0, 0]
        else:
            tn, fp, fn, tp = cm.ravel()
        
        # 计算各项指标
        sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
        specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
        ppv = tp / (tp + fp) if (tp + fp) > 0 else 0
        npv = tn / (tn + fn) if (tn + fn) > 0 else 0
        lr_pos = sensitivity / (1 - specificity) if (1 - specificity) > 0 else float('inf')
        lr_neg = (1 - sensitivity) / specificity if specificity > 0 else float('inf')
        
        # 存储结果
        metrics[cls] = {
            'Sensitivity': sensitivity,
            'Specificity': specificity,
            'PPV': ppv,
            'NPV': npv,
            'LR+': lr_pos,
            'LR-': lr_neg
        }
    
    # 计算总体准确率
    accuracy = accuracy_score(y_true, y_pred)
    metrics['overall'] = {'Accuracy': accuracy}
    
    return metrics

# 获取各模型在测试集上的预测结果和预测概率（使用筛选后的特征）
y_pred_rf = clf_rf.predict(X_test_selected)
y_pred_proba_rf = clf_rf.predict_proba(X_test_selected)
y_pred_dt = clf_dt.predict(X_test_selected)
y_pred_proba_dt = clf_dt.predict_proba(X_test_selected)
y_pred_lg = clf_lg.predict(X_test_selected)
y_pred_proba_lg = clf_lg.predict_proba(X_test_selected)
y_pred_svm = clf_svm.predict(X_test_selected)
y_pred_proba_svm = clf_svm.predict_proba(X_test_selected)
y_pred_gbm = clf_gbm.predict(X_test_selected)
y_pred_proba_gbm = clf_gbm.predict_proba(X_test_selected)
y_pred_xgb = clf_xgb.predict(X_test_selected)
y_pred_proba_xgb = clf_xgb.predict_proba(X_test_selected)
y_pred_stacking = stacking_clf.predict(X_test_selected)
y_pred_proba_stacking = stacking_clf.predict_proba(X_test_selected)

# 获取各模型在训练集上的预测结果和预测概率
y_pred_train_rf = clf_rf.predict(X_train_selected)
y_pred_proba_train_rf = clf_rf.predict_proba(X_train_selected)
y_pred_train_dt = clf_dt.predict(X_train_selected)
y_pred_proba_train_dt = clf_dt.predict_proba(X_train_selected)
y_pred_train_lg = clf_lg.predict(X_train_selected)
y_pred_proba_train_lg = clf_lg.predict_proba(X_train_selected)
y_pred_train_svm = clf_svm.predict(X_train_selected)
y_pred_proba_train_svm = clf_svm.predict_proba(X_train_selected)
y_pred_train_gbm = clf_gbm.predict(X_train_selected)
y_pred_proba_train_gbm = clf_gbm.predict_proba(X_train_selected)
y_pred_train_xgb = clf_xgb.predict(X_train_selected)
y_pred_proba_train_xgb = clf_xgb.predict_proba(X_train_selected)
y_pred_train_stacking = stacking_clf.predict(X_train_selected)
y_pred_proba_train_stacking = stacking_clf.predict_proba(X_train_selected)

# 计算各模型的 ROC 曲线及 AUC 值 - 测试集
fpr_rf, tpr_rf, roc_auc_rf, roc_auc_ci_rf = calculate_roc_auc('RandomForest', y_pred_proba_rf, y_test)
fpr_dt, tpr_dt, roc_auc_dt, roc_auc_ci_dt = calculate_roc_auc('DecisionTree', y_pred_proba_dt, y_test)
fpr_lg, tpr_lg, roc_auc_lg, roc_auc_ci_lg = calculate_roc_auc('LogisticRegression', y_pred_proba_lg, y_test)
fpr_svm, tpr_svm, roc_auc_svm, roc_auc_ci_svm = calculate_roc_auc('SVM', y_pred_proba_svm, y_test)
fpr_gbm, tpr_gbm, roc_auc_gbm, roc_auc_ci_gbm = calculate_roc_auc('GBM', y_pred_proba_gbm, y_test)
fpr_xgb, tpr_xgb, roc_auc_xgb, roc_auc_ci_xgb = calculate_roc_auc('XGBoost', y_pred_proba_xgb, y_test)
fpr_stacking, tpr_stacking, roc_auc_stacking, roc_auc_ci_stacking = calculate_roc_auc('StackingClassifier', y_pred_proba_stacking, y_test)

# 计算各模型的 ROC 曲线及 AUC 值 - 训练集
fpr_train_rf, tpr_train_rf, roc_auc_train_rf, roc_auc_ci_train_rf = calculate_roc_auc('RandomForest', y_pred_proba_train_rf, y_train)
fpr_train_dt, tpr_train_dt, roc_auc_train_dt, roc_auc_ci_train_dt = calculate_roc_auc('DecisionTree', y_pred_proba_train_dt, y_train)
fpr_train_lg, tpr_train_lg, roc_auc_train_lg, roc_auc_ci_train_lg = calculate_roc_auc('LogisticRegression', y_pred_proba_train_lg, y_train)
fpr_train_svm, tpr_train_svm, roc_auc_train_svm, roc_auc_ci_train_svm = calculate_roc_auc('SVM', y_pred_proba_train_svm, y_train)
fpr_train_gbm, tpr_train_gbm, roc_auc_train_gbm, roc_auc_ci_train_gbm = calculate_roc_auc('GBM', y_pred_proba_train_gbm, y_train)
fpr_train_xgb, tpr_train_xgb, roc_auc_train_xgb, roc_auc_ci_train_xgb = calculate_roc_auc('XGBoost', y_pred_proba_train_xgb, y_train)
fpr_train_stacking, tpr_train_stacking, roc_auc_train_stacking, roc_auc_ci_train_stacking = calculate_roc_auc('StackingClassifier', y_pred_proba_train_stacking, y_train)

# 绘制测试集上所有类别的 ROC 曲线
plt.figure(figsize=(12, 8))
line_styles = ['-', '--', '-.', ':']
colors = ['r', 'g', 'b', 'c', 'm', 'y', 'k']
markers = ['o', 's', '^', '*', 'D', 'X', 'v']

# 获取唯一的类别列表
unique_classes = np.unique(y_test)
num_classes = len(unique_classes)

# 多分类问题，为每个类别绘制所有模型的曲线
for group_idx, group in enumerate(unique_classes):
    # 随机森林模型的曲线设置
    plt.plot(fpr_rf[group], tpr_rf[group], label=f'Random Forest -  {group}',
             lw=2, c=colors[0], marker=markers[0], linestyle=line_styles[group_idx % len(line_styles)])
    # 决策树模型的曲线设置
    plt.plot(fpr_dt[group], tpr_dt[group], label=f'Decision Tree -  {group}',
             lw=2, c=colors[1], marker=markers[1], linestyle=line_styles[group_idx % len(line_styles)])
    # 逻辑回归模型的曲线设置
    plt.plot(fpr_lg[group], tpr_lg[group], label=f'LogisticRegression -  {group}',
             lw=2, c=colors[2], marker=markers[2], linestyle=line_styles[group_idx % len(line_styles)])
    # SVM模型的曲线设置
    plt.plot(fpr_svm[group], tpr_svm[group], label=f'SVM -  {group}',
             lw=2, c=colors[3], marker=markers[3], linestyle=line_styles[group_idx % len(line_styles)])
    # GBM模型的曲线设置
    plt.plot(fpr_gbm[group], tpr_gbm[group], 
             label=f'GBM -  {group}',
             lw=2, c=colors[4], marker=markers[4], 
             linestyle=line_styles[group_idx % len(line_styles)])
    # XGBoost 模型的曲线设置
    plt.plot(fpr_xgb[group], tpr_xgb[group], 
             label=f'XGBoost -  {group}',
             lw=2, c=colors[5], marker=markers[5], 
             linestyle=line_styles[group_idx % len(line_styles)])
    # 堆叠分类器模型的曲线设置
    plt.plot(fpr_stacking[group], tpr_stacking[group], 
             label=f'StackingClassifier -  {group}',
             lw=2, c=colors[6], marker=markers[6], 
             linestyle=line_styles[group_idx % len(line_styles)])

plt.plot([0, 1], [0, 1], 'k--', lw=2)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('1-Specificity')
plt.ylabel('Sensitivity')
#plt.title('ROC Curves for All Classes (Test Set)')
plt.legend(loc="lower right", fontsize='small')
plt.grid(True, which='both', linestyle='--', linewidth=0.5)
plt.savefig('./LF-SML-roc(test).tif', dpi=300)
plt.show()

# 绘制训练集上所有类别的 ROC 曲线
plt.figure(figsize=(12, 8))
line_styles = ['-', '--', '-.', ':']
colors = ['r', 'g', 'b', 'c', 'm', 'y', 'k']
markers = ['o', 's', '^', '*', 'D', 'X', 'v']

# 获取唯一的类别列表
unique_classes = np.unique(y_train)
num_classes = len(unique_classes)

# 多分类问题，为每个类别绘制所有模型的曲线
for group_idx, group in enumerate(unique_classes):
    # 随机森林模型的曲线设置
    plt.plot(fpr_train_rf[group], tpr_train_rf[group], label=f'Random Forest -  {group}',
             lw=2, c=colors[0], marker=markers[0], linestyle=line_styles[group_idx % len(line_styles)])
    # 决策树模型的曲线设置
    plt.plot(fpr_train_dt[group], tpr_train_dt[group], label=f'Decision Tree -  {group}',
             lw=2, c=colors[1], marker=markers[1], linestyle=line_styles[group_idx % len(line_styles)])
    # 逻辑回归模型的曲线设置
    plt.plot(fpr_train_lg[group], tpr_train_lg[group], label=f'LogisticRegression -  {group}',
             lw=2, c=colors[2], marker=markers[2], linestyle=line_styles[group_idx % len(line_styles)])
    # SVM模型的曲线设置
    plt.plot(fpr_train_svm[group], tpr_train_svm[group], label=f'SVM -  {group}',
             lw=2, c=colors[3], marker=markers[3], linestyle=line_styles[group_idx % len(line_styles)])
    # GBM模型的曲线设置
    plt.plot(fpr_train_gbm[group], tpr_train_gbm[group], 
             label=f'GBM -  {group}',
             lw=2, c=colors[4], marker=markers[4], 
             linestyle=line_styles[group_idx % len(line_styles)])
    # XGBoost 模型的曲线设置
    plt.plot(fpr_train_xgb[group], tpr_train_xgb[group], 
             label=f'XGBoost -  {group}',
             lw=2, c=colors[5], marker=markers[5], 
             linestyle=line_styles[group_idx % len(line_styles)])
    # 堆叠分类器模型的曲线设置
    plt.plot(fpr_train_stacking[group], tpr_train_stacking[group], 
             label=f'StackingClassifier -  {group}',
             lw=2, c=colors[6], marker=markers[6], 
             linestyle=line_styles[group_idx % len(line_styles)])

plt.plot([0, 1], [0, 1], 'k--', lw=2)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('1-Specificity')
plt.ylabel('Sensitivity')
#plt.title('ROC Curves for All Classes (Training Set)')
plt.legend(loc="lower right", fontsize='small')
plt.grid(True, which='both', linestyle='--', linewidth=0.5)
plt.savefig('./LF-SML-roc(train).tif', dpi=300)
plt.show()

# 计算并打印各模型的性能指标
def print_model_metrics(model_name, y_true, y_pred, dataset_type):
    metrics = calculate_metrics(y_true, y_pred)
    
    print(f"\n{model_name} Model ({dataset_type}):")
    
    # 获取所有类别键（排除'overall'）
    class_keys = [k for k in metrics.keys() if k != 'overall']
    
    # 尝试对类别键进行排序
    try:
        # 尝试数值排序
        class_keys = sorted(class_keys, key=lambda x: int(x))
    except (ValueError, TypeError):
        # 如果无法数值排序，则进行字符串排序
        class_keys = sorted(class_keys, key=lambda x: str(x))
    
    # 打印每个类别的指标
    for cls in class_keys:
        print(f"\n  类别 {cls}:")
        print(f"    Sensitivity: {metrics[cls]['Sensitivity']:.4f}")
        print(f"    Specificity: {metrics[cls]['Specificity']:.4f}")
        print(f"    Positive Predictive Value (PPV): {metrics[cls]['PPV']:.4f}")
        print(f"    Negative Predictive Value (NPV): {metrics[cls]['NPV']:.4f}")
        print(f"    Positive Likelihood Ratio (LR+): {metrics[cls]['LR+']:.4f}")
        print(f"    Negative Likelihood Ratio (LR-): {metrics[cls]['LR-']:.4f}")
    
    # 打印总体准确率
    print(f"\n  总体准确率: {metrics['overall']['Accuracy']:.4f}")

# 打印测试集指标
print_model_metrics("Random Forest", y_test, y_pred_rf, "Test Set")
print_model_metrics("Decision Tree", y_test, y_pred_dt, "Test Set")
print_model_metrics("Logistic Regression", y_test, y_pred_lg, "Test Set")
print_model_metrics("SVM", y_test, y_pred_svm, "Test Set")
print_model_metrics("Gradient Boosting", y_test, y_pred_gbm, "Test Set")
print_model_metrics("XGBoost", y_test, y_pred_xgb, "Test Set")
print_model_metrics("Stacking Classifier", y_test, y_pred_stacking, "Test Set")

# 打印训练集指标
print_model_metrics("Random Forest", y_train, y_pred_train_rf, "Training Set")
print_model_metrics("Decision Tree", y_train, y_pred_train_dt, "Training Set")
print_model_metrics("Logistic Regression", y_train, y_pred_train_lg, "Training Set")
print_model_metrics("SVM", y_train, y_pred_train_svm, "Training Set")
print_model_metrics("Gradient Boosting", y_train, y_pred_train_gbm, "Training Set")
print_model_metrics("XGBoost", y_train, y_pred_train_xgb, "Training Set")
print_model_metrics("Stacking Classifier", y_train, y_pred_train_stacking, "Training Set")

# 保存所有模型的测试集结果，按分类保存
for model_name, y_pred_proba, y_pred in zip(
    ['RandomForest', 'DecisionTree', 'LogisticRegression', 'SVM', 'GBM', 'XGBoost', 'StackingClassifier'],
    [y_pred_proba_rf, y_pred_proba_dt, y_pred_proba_lg, y_pred_proba_svm, y_pred_proba_gbm, y_pred_proba_xgb, y_pred_proba_stacking],
    [y_pred_rf, y_pred_dt, y_pred_lg, y_pred_svm, y_pred_gbm, y_pred_xgb, y_pred_stacking]
):
    # 创建模型目录
    model_dir = os.path.join(results_dir, "test", model_name)
    os.makedirs(model_dir, exist_ok=True)
    
    # 保存原始预测结果
    custom_name = f'LF-SML-{model_name}(test)'
    np.save(os.path.join(model_dir, f'{custom_name}_y_true.npy'), y_test)
    np.save(os.path.join(model_dir, f'{custom_name}_y_proba.npy'), y_pred_proba)
    np.save(os.path.join(model_dir, f'{custom_name}_y_pred.npy'), y_pred)
    print(f"已保存 {custom_name} 模型的测试集原始结果")
    
    # 按分类保存预测结果
    for class_idx, class_name in enumerate(unique_classes):
        class_dir = os.path.join(model_dir, f"class_{class_name}")
        os.makedirs(class_dir, exist_ok=True)
        
        # 为当前类别创建二分类标签
        y_true_binary = (y_test == class_name).astype(int)
        y_pred_binary = (y_pred == class_name).astype(int)
        y_proba_binary = y_pred_proba[:, class_idx]
        
        # 保存二分类结果
        np.save(os.path.join(class_dir, f'{custom_name}_class{class_name}_y_true.npy'), y_true_binary)
        np.save(os.path.join(class_dir, f'{custom_name}_class{class_name}_y_proba.npy'), y_proba_binary)
        np.save(os.path.join(class_dir, f'{custom_name}_class{class_name}_y_pred.npy'), y_pred_binary)
        print(f"已保存 {custom_name} 模型类别 {class_name} 的测试集二分类结果")

# 保存所有模型的训练集结果，按分类保存
for model_name, y_pred_proba, y_pred in zip(
    ['RandomForest', 'DecisionTree', 'LogisticRegression', 'SVM', 'GBM', 'XGBoost', 'StackingClassifier'],
    [y_pred_proba_train_rf, y_pred_proba_train_dt, y_pred_proba_train_lg, y_pred_proba_train_svm, y_pred_proba_train_gbm, y_pred_proba_train_xgb, y_pred_proba_train_stacking],
    [y_pred_train_rf, y_pred_train_dt, y_pred_train_lg, y_pred_train_svm, y_pred_train_gbm, y_pred_train_xgb, y_pred_train_stacking]
):
    # 创建模型目录
    model_dir = os.path.join(results_dir, "train", model_name)
    os.makedirs(model_dir, exist_ok=True)
    
    # 保存原始预测结果
    custom_name = f'LF-SML-{model_name}(train)'
    np.save(os.path.join(model_dir, f'{custom_name}_y_true.npy'), y_train)
    np.save(os.path.join(model_dir, f'{custom_name}_y_proba.npy'), y_pred_proba)
    np.save(os.path.join(model_dir, f'{custom_name}_y_pred.npy'), y_pred)
    print(f"已保存 {custom_name} 模型的训练集原始结果")
    
    # 按分类保存预测结果
    for class_idx, class_name in enumerate(unique_classes):
        class_dir = os.path.join(model_dir, f"class_{class_name}")
        os.makedirs(class_dir, exist_ok=True)
        
        # 为当前类别创建二分类标签
        y_true_binary = (y_train == class_name).astype(int)
        y_pred_binary = (y_pred == class_name).astype(int)
        y_proba_binary = y_pred_proba[:, class_idx]
        
        # 保存二分类结果
        np.save(os.path.join(class_dir, f'{custom_name}_class{class_name}_y_true.npy'), y_true_binary)
        np.save(os.path.join(class_dir, f'{custom_name}_class{class_name}_y_proba.npy'), y_proba_binary)
        np.save(os.path.join(class_dir, f'{custom_name}_class{class_name}_y_pred.npy'), y_pred_binary)
        print(f"已保存 {custom_name} 模型类别 {class_name} 的训练集二分类结果")
